Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '../../Data/Tutorial/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.517204
Minibatch accuracy: 31.2%
Validation accuracy: 10.3%
Minibatch loss at step 50: 1.727875
Minibatch accuracy: 50.0%
Validation accuracy: 61.3%
Minibatch loss at step 100: 1.010757
Minibatch accuracy: 75.0%
Validation accuracy: 72.6%
Minibatch loss at step 150: 0.429464
Minibatch accuracy: 87.5%
Validation accuracy: 75.9%
Minibatch loss at step 200: 0.416449
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.666303
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.990121
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.573856
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.791242
Minibatch accuracy: 75.0%
Validation accuracy: 80.9%
Minibatch loss at step 450: 0.735004
Minibatch accuracy: 93.8%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.435285
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

**Building model for max pooling**

In [20]:
def printShape(layer) :
    print (layer.get_shape().as_list())

In [40]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(value=conv,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool_1')
    hidden = tf.nn.relu(pool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(value=conv,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool_2')
    hidden = tf.nn.relu(pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Run the graph having modified with max_pooling

In [57]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.788230
Minibatch accuracy: 18.8%
Validation accuracy: 10.9%
Minibatch loss at step 50: 1.618340
Minibatch accuracy: 37.5%
Validation accuracy: 62.2%
Minibatch loss at step 100: 0.927144
Minibatch accuracy: 75.0%
Validation accuracy: 73.6%
Minibatch loss at step 150: 0.324574
Minibatch accuracy: 87.5%
Validation accuracy: 75.7%
Minibatch loss at step 200: 0.249898
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 250: 0.641651
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 300: 0.798710
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.765651
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
Minibatch loss at step 400: 0.573116
Minibatch accuracy: 75.0%
Validation accuracy: 82.6%
Minibatch loss at step 450: 0.541602
Minibatch accuracy: 93.8%
Validation accuracy: 82.3%
Minibatch loss at step 500: 0.473925
Minibatch accuracy: 81.2%
Validation accuracy: 83.1%
M

Minibatch loss at step 4550: 0.048767
Minibatch accuracy: 100.0%
Validation accuracy: 88.0%
Minibatch loss at step 4600: 0.367138
Minibatch accuracy: 81.2%
Validation accuracy: 88.3%
Minibatch loss at step 4650: 0.417132
Minibatch accuracy: 81.2%
Validation accuracy: 88.3%
Minibatch loss at step 4700: 0.071369
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at step 4750: 0.033148
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at step 4800: 0.728334
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 4850: 0.462292
Minibatch accuracy: 81.2%
Validation accuracy: 88.0%
Minibatch loss at step 4900: 0.546218
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 4950: 0.027241
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at step 5000: 0.293200
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 5050: 0.634194
Minibatch accuracy: 81.2%
Validation accuracy: 8

Minibatch loss at step 9050: 0.258961
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Minibatch loss at step 9100: 0.451310
Minibatch accuracy: 81.2%
Validation accuracy: 88.9%
Minibatch loss at step 9150: 0.568596
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 9200: 0.374425
Minibatch accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss at step 9250: 0.154762
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 9300: 0.353819
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 9350: 0.638606
Minibatch accuracy: 87.5%
Validation accuracy: 89.4%
Minibatch loss at step 9400: 0.516397
Minibatch accuracy: 87.5%
Validation accuracy: 89.5%
Minibatch loss at step 9450: 0.969762
Minibatch accuracy: 68.8%
Validation accuracy: 89.4%
Minibatch loss at step 9500: 0.061781
Minibatch accuracy: 100.0%
Validation accuracy: 89.4%
Minibatch loss at step 9550: 0.713649
Minibatch accuracy: 81.2%
Validation accuracy: 88.7

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Adding Dropout and learning rate decay

In [67]:
batch_size = 128
patch_size = 3
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, training = False):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(value=conv,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool_1')
    hidden = tf.nn.relu(pool + layer1_biases)
    if (training) :
        hidden = tf.nn.dropout(hidden, keep_prob=0.8)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(value=conv,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool_2')
    hidden = tf.nn.relu(pool + layer2_biases)
    if (training) :
        hidden = tf.nn.dropout(hidden, keep_prob=0.8)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)

  #calculate l2 regularization loss
  vars   = tf.trainable_variables()
  lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in vars if 'bias' not in v.name ]) * 0.001

  #add l2 regularization loss to actual loss, so that optimizer finds a way to minimize it too.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + lossL2
    
  #Learning rate.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    1000, 0.95, staircase=True)
  
  # Optimizer. 
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [69]:
num_steps = 50001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, lr, l, predictions = session.run(
      [optimizer, learning_rate, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f at learning rate : %f' % (step, l, lr))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.020921 at learning rate : 0.100000
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.540309 at learning rate : 0.100000
Minibatch accuracy: 12.5%
Validation accuracy: 14.7%
Minibatch loss at step 100: 2.450184 at learning rate : 0.100000
Minibatch accuracy: 27.3%
Validation accuracy: 19.0%
Minibatch loss at step 150: 1.431994 at learning rate : 0.100000
Minibatch accuracy: 63.3%
Validation accuracy: 61.8%
Minibatch loss at step 200: 1.117795 at learning rate : 0.100000
Minibatch accuracy: 74.2%
Validation accuracy: 75.0%
Minibatch loss at step 250: 1.204517 at learning rate : 0.100000
Minibatch accuracy: 71.9%
Validation accuracy: 76.7%
Minibatch loss at step 300: 0.942208 at learning rate : 0.100000
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Minibatch loss at step 350: 0.989900 at learning rate : 0.100000
Minibatch accuracy: 77.3%
Validation accuracy: 80.6%
Minibatch loss at step 400: 0.826768 at learnin

Minibatch loss at step 3450: 0.359164 at learning rate : 0.085737
Minibatch accuracy: 96.1%
Validation accuracy: 89.2%
Minibatch loss at step 3500: 0.630616 at learning rate : 0.085737
Minibatch accuracy: 92.2%
Validation accuracy: 88.8%
Minibatch loss at step 3550: 0.628462 at learning rate : 0.085737
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 3600: 0.497066 at learning rate : 0.085737
Minibatch accuracy: 91.4%
Validation accuracy: 89.0%
Minibatch loss at step 3650: 0.462913 at learning rate : 0.085737
Minibatch accuracy: 93.0%
Validation accuracy: 89.1%
Minibatch loss at step 3700: 0.443411 at learning rate : 0.085737
Minibatch accuracy: 92.2%
Validation accuracy: 89.2%
Minibatch loss at step 3750: 0.470622 at learning rate : 0.085737
Minibatch accuracy: 92.2%
Validation accuracy: 88.7%
Minibatch loss at step 3800: 0.518681 at learning rate : 0.085737
Minibatch accuracy: 88.3%
Validation accuracy: 88.7%
Minibatch loss at step 3850: 0.469546 at learnin

Minibatch loss at step 6900: 0.552008 at learning rate : 0.073509
Minibatch accuracy: 86.7%
Validation accuracy: 89.9%
Minibatch loss at step 6950: 0.416911 at learning rate : 0.073509
Minibatch accuracy: 89.8%
Validation accuracy: 90.2%
Minibatch loss at step 7000: 0.390473 at learning rate : 0.069834
Minibatch accuracy: 91.4%
Validation accuracy: 90.0%
Minibatch loss at step 7050: 0.344024 at learning rate : 0.069834
Minibatch accuracy: 94.5%
Validation accuracy: 89.9%
Minibatch loss at step 7100: 0.487739 at learning rate : 0.069834
Minibatch accuracy: 89.8%
Validation accuracy: 90.1%
Minibatch loss at step 7150: 0.341999 at learning rate : 0.069834
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 7200: 0.373220 at learning rate : 0.069834
Minibatch accuracy: 91.4%
Validation accuracy: 90.2%
Minibatch loss at step 7250: 0.311642 at learning rate : 0.069834
Minibatch accuracy: 95.3%
Validation accuracy: 89.8%
Minibatch loss at step 7300: 0.358715 at learnin

Minibatch loss at step 10350: 0.400117 at learning rate : 0.059874
Minibatch accuracy: 92.2%
Validation accuracy: 90.7%
Minibatch loss at step 10400: 0.478811 at learning rate : 0.059874
Minibatch accuracy: 85.9%
Validation accuracy: 90.1%
Minibatch loss at step 10450: 0.287003 at learning rate : 0.059874
Minibatch accuracy: 96.1%
Validation accuracy: 90.6%
Minibatch loss at step 10500: 0.317696 at learning rate : 0.059874
Minibatch accuracy: 93.0%
Validation accuracy: 90.3%
Minibatch loss at step 10550: 0.458849 at learning rate : 0.059874
Minibatch accuracy: 90.6%
Validation accuracy: 90.4%
Minibatch loss at step 10600: 0.398031 at learning rate : 0.059874
Minibatch accuracy: 90.6%
Validation accuracy: 90.3%
Minibatch loss at step 10650: 0.563124 at learning rate : 0.059874
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 10700: 0.373856 at learning rate : 0.059874
Minibatch accuracy: 92.2%
Validation accuracy: 90.4%
Minibatch loss at step 10750: 0.493977 a

Minibatch loss at step 13800: 0.273770 at learning rate : 0.051334
Minibatch accuracy: 96.1%
Validation accuracy: 90.5%
Minibatch loss at step 13850: 0.310294 at learning rate : 0.051334
Minibatch accuracy: 93.0%
Validation accuracy: 90.5%
Minibatch loss at step 13900: 0.450988 at learning rate : 0.051334
Minibatch accuracy: 89.1%
Validation accuracy: 90.9%
Minibatch loss at step 13950: 0.311505 at learning rate : 0.051334
Minibatch accuracy: 94.5%
Validation accuracy: 90.7%
Minibatch loss at step 14000: 0.382004 at learning rate : 0.048767
Minibatch accuracy: 89.1%
Validation accuracy: 90.7%
Minibatch loss at step 14050: 0.312912 at learning rate : 0.048767
Minibatch accuracy: 93.8%
Validation accuracy: 90.6%
Minibatch loss at step 14100: 0.331354 at learning rate : 0.048767
Minibatch accuracy: 93.8%
Validation accuracy: 90.7%
Minibatch loss at step 14150: 0.396333 at learning rate : 0.048767
Minibatch accuracy: 91.4%
Validation accuracy: 90.6%
Minibatch loss at step 14200: 0.355768 a

...

Minibatch loss at step 34500: 0.356006 at learning rate : 0.017482
Minibatch accuracy: 90.6%
Validation accuracy: 91.1%
Minibatch loss at step 34550: 0.347083 at learning rate : 0.017482
Minibatch accuracy: 92.2%
Validation accuracy: 91.4%
Minibatch loss at step 34600: 0.376984 at learning rate : 0.017482
Minibatch accuracy: 88.3%
Validation accuracy: 91.3%
Minibatch loss at step 34650: 0.385139 at learning rate : 0.017482
Minibatch accuracy: 89.8%
Validation accuracy: 91.4%
Minibatch loss at step 34700: 0.283997 at learning rate : 0.017482
Minibatch accuracy: 96.1%
Validation accuracy: 91.5%
Minibatch loss at step 34750: 0.376867 at learning rate : 0.017482
Minibatch accuracy: 90.6%
Validation accuracy: 91.4%
Minibatch loss at step 34800: 0.313299 at learning rate : 0.017482
Minibatch accuracy: 92.2%
Validation accuracy: 91.4%
Minibatch loss at step 34850: 0.391187 at learning rate : 0.017482
Minibatch accuracy: 94.5%
Validation accuracy: 91.3%
Minibatch loss at step 34900: 0.338255 a

Minibatch loss at step 37950: 0.212917 at learning rate : 0.014989
Minibatch accuracy: 94.5%
Validation accuracy: 91.3%
Minibatch loss at step 38000: 0.312361 at learning rate : 0.014240
Minibatch accuracy: 92.2%
Validation accuracy: 91.4%
Minibatch loss at step 38050: 0.374807 at learning rate : 0.014240
Minibatch accuracy: 92.2%
Validation accuracy: 91.5%
Minibatch loss at step 38100: 0.260291 at learning rate : 0.014240
Minibatch accuracy: 93.8%
Validation accuracy: 91.5%
Minibatch loss at step 38150: 0.248762 at learning rate : 0.014240
Minibatch accuracy: 96.1%
Validation accuracy: 91.5%
Minibatch loss at step 38200: 0.386178 at learning rate : 0.014240
Minibatch accuracy: 91.4%
Validation accuracy: 91.5%
Minibatch loss at step 38250: 0.351242 at learning rate : 0.014240
Minibatch accuracy: 92.2%
Validation accuracy: 91.5%
Minibatch loss at step 38300: 0.368357 at learning rate : 0.014240
Minibatch accuracy: 89.1%
Validation accuracy: 91.5%
Minibatch loss at step 38350: 0.239645 a

Minibatch loss at step 41400: 0.217340 at learning rate : 0.012209
Minibatch accuracy: 93.8%
Validation accuracy: 91.5%
Minibatch loss at step 41450: 0.352735 at learning rate : 0.012209
Minibatch accuracy: 92.2%
Validation accuracy: 91.6%
Minibatch loss at step 41500: 0.369641 at learning rate : 0.012209
Minibatch accuracy: 93.0%
Validation accuracy: 91.8%
Minibatch loss at step 41550: 0.195361 at learning rate : 0.012209
Minibatch accuracy: 96.1%
Validation accuracy: 91.7%
Minibatch loss at step 41600: 0.306112 at learning rate : 0.012209
Minibatch accuracy: 93.0%
Validation accuracy: 91.5%
Minibatch loss at step 41650: 0.216541 at learning rate : 0.012209
Minibatch accuracy: 96.1%
Validation accuracy: 91.5%
Minibatch loss at step 41700: 0.264834 at learning rate : 0.012209
Minibatch accuracy: 95.3%
Validation accuracy: 91.5%
Minibatch loss at step 41750: 0.286630 at learning rate : 0.012209
Minibatch accuracy: 93.0%
Validation accuracy: 91.6%
Minibatch loss at step 41800: 0.265269 a

Minibatch loss at step 44850: 0.270620 at learning rate : 0.010467
Minibatch accuracy: 93.8%
Validation accuracy: 91.5%
Minibatch loss at step 44900: 0.252527 at learning rate : 0.010467
Minibatch accuracy: 94.5%
Validation accuracy: 91.5%
Minibatch loss at step 44950: 0.319843 at learning rate : 0.010467
Minibatch accuracy: 93.0%
Validation accuracy: 91.3%
Minibatch loss at step 45000: 0.208432 at learning rate : 0.009944
Minibatch accuracy: 95.3%
Validation accuracy: 91.6%
Minibatch loss at step 45050: 0.275190 at learning rate : 0.009944
Minibatch accuracy: 93.0%
Validation accuracy: 91.6%
Minibatch loss at step 45100: 0.421612 at learning rate : 0.009944
Minibatch accuracy: 89.1%
Validation accuracy: 91.7%
Minibatch loss at step 45150: 0.233933 at learning rate : 0.009944
Minibatch accuracy: 95.3%
Validation accuracy: 91.5%
Minibatch loss at step 45200: 0.316874 at learning rate : 0.009944
Minibatch accuracy: 89.1%
Validation accuracy: 91.6%
Minibatch loss at step 45250: 0.374763 a

Minibatch loss at step 48300: 0.384720 at learning rate : 0.008526
Minibatch accuracy: 91.4%
Validation accuracy: 91.5%
Minibatch loss at step 48350: 0.310795 at learning rate : 0.008526
Minibatch accuracy: 91.4%
Validation accuracy: 91.6%
Minibatch loss at step 48400: 0.283756 at learning rate : 0.008526
Minibatch accuracy: 91.4%
Validation accuracy: 91.5%
Minibatch loss at step 48450: 0.349162 at learning rate : 0.008526
Minibatch accuracy: 92.2%
Validation accuracy: 91.6%
Minibatch loss at step 48500: 0.344054 at learning rate : 0.008526
Minibatch accuracy: 93.8%
Validation accuracy: 91.4%
Minibatch loss at step 48550: 0.273789 at learning rate : 0.008526
Minibatch accuracy: 93.8%
Validation accuracy: 91.5%
Minibatch loss at step 48600: 0.324983 at learning rate : 0.008526
Minibatch accuracy: 93.0%
Validation accuracy: 91.6%
Minibatch loss at step 48650: 0.213526 at learning rate : 0.008526
Minibatch accuracy: 95.3%
Validation accuracy: 91.7%
Minibatch loss at step 48700: 0.382887 a

Best so far 96.3% on test set. But using conv net I expected more. Let's see on the next one. I will come back to this.